In [265]:
import os
import sys
import pyspark
from pyspark.sql.window import *
from pyspark.sql.functions import *
import operator
from pyspark.sql import SparkSession
from pyspark import SQLContext


In [396]:
#df = op.load.csv("C:\\Users\\Ajibola Vincent\\Documents\\Udacity\\ay1.csv")
#sqlContext = SQLContext(sc)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv("C:\\Users\\Ajibola Vincent\\Documents\\Udacity\\ay1.csv",header=True,sep=",");
sqlContext = SQLContext(spark)
df.count()


506

In [397]:
df = df.select('account_id','account_nbr').na.drop(subset=["account_nbr"])
df.show()
w = Window.partitionBy('account_id').orderBy('account_nbr')

+----------+-----------+
|account_id|account_nbr|
+----------+-----------+
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|   66487163|
|         2|          0|
|         2|   66487163|
|         2|   13943797|
|         2|          0|
|         2|   66487163|
|         2|   13943797|
|         2|   66487163|
|         2|          0|
|         2|          0|
|         2|   13943797|
|         2|   66487163|
|         2|   13943797|
+----------+-----------+
only showing top 20 rows



In [398]:
df = df.withColumn('sorted_list', collect_set('account_nbr').over(w))\
        .groupBy('account_id')\
        .agg(max('sorted_list').alias('sorted_list'))

In [399]:
df.show()

+----------+--------------------+
|account_id|         sorted_list|
+----------+--------------------+
|        15|          [24265569]|
|        34|[19993174, 589990...|
|        22|          [97847811]|
|        35|[99848401, 567886...|
|        31|          [12345679]|
|        18|          [14567877]|
|      2357|          [93012204]|
|        27|          [28175699]|
|        26|[12891853, 73418420]|
|        23|          [20774019]|
|      3818|          [86543521]|
|      2177|          [62457513]|
|        33|          [33047658]|
|         9|          [13943797]|
|       485| [4968433, 42968433]|
|         2|[89597016, 664871...|
|      1844|[42988401, 8, 429...|
+----------+--------------------+



In [400]:
df.withColumn(
    'sorted_list', df["sorted_list"].getItem(0)
).show(truncate=False)


+----------+-----------+
|account_id|sorted_list|
+----------+-----------+
|15        |24265569   |
|34        |19993174   |
|22        |97847811   |
|35        |99848401   |
|31        |12345679   |
|18        |14567877   |
|2357      |93012204   |
|27        |28175699   |
|26        |12891853   |
|23        |20774019   |
|3818      |86543521   |
|2177      |62457513   |
|33        |33047658   |
|9         |13943797   |
|485       |4968433    |
|2         |89597016   |
|1844      |42988401   |
+----------+-----------+

